# Analysis of multigrid restriction and prolongation
1D Poisson problem, discretized with a second order finite difference formula, yields the following linear system

In [39]:
import sympy as sym
sym.init_printing()


h = sym.symbols('h')
n = 17
m = 1 + (n-1)//2


def build_poisson_operator_1d(n):
    A = sym.zeros(n,n)
    
    A[0,0] = 1
    A[n-1,n-1] = 1
    
    for i in range(1,n-1):
        A[i,i-1] = -1 / h**2
        A[i,i]   =  2 / h**2
        A[i,i+1] = -1 / h**2
        
    return A

A = build_poisson_operator_1d(n)
A

⎡ 1    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0 
⎢                                                                                   
⎢-1   2    -1                                                                       
⎢───  ──   ───   0    0    0    0    0    0    0    0    0    0    0    0    0    0 
⎢  2   2     2                                                                      
⎢ h   h     h                                                                       
⎢                                                                                   
⎢     -1   2    -1                                                                  
⎢ 0   ───  ──   ───   0    0    0    0    0    0    0    0    0    0    0    0    0 
⎢       2   2     2                                                                 
⎢      h   h     h                                                                  
⎢                                                                

The projection on a coarser grid is supposed to work on smooth error components. As for "A multigrid tutorial" the operator for relaxing on the coarser grid is:

$$ A^{2h} := I_{h}^{2h}A^{h}I_{2h}^{h} $$

There is freedom in determining the restriction and prolongation operators, but it is advisable that

$$ I_{h}^{2h} = c(I_{2h}^{h})^{T} $$

A careful choice results in $A^{2h}$ being the matrix of the original problem, discretized on a grid double the size. That is desirable because it simplifies the solver



## Naive choices

In [40]:
def injective_restriction_1d(n):
    m = 1 + (n-1)//2
    R = sym.zeros(m,n)
    
    for i in range(m):
        R[i,2*i] = 1
        
    return R

injective_restriction_1d(n)

⎡1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0⎤
⎢                                                 ⎥
⎢0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0⎥
⎢                                                 ⎥
⎢0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0⎥
⎢                                                 ⎥
⎢0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0⎥
⎢                                                 ⎥
⎢0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0⎥
⎢                                                 ⎥
⎢0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0⎥
⎢                                                 ⎥
⎢0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0⎥
⎢                                                 ⎥
⎢0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0⎥
⎢                                                 ⎥
⎣0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1⎦

In [41]:
def linear_prolongation_1d(m):
    n = 2 * (m - 1) + 1
    P = sym.zeros(n,m)
    
    for i in range(n):
        if i % 2 == 0:
            P[i,i//2] = 1
        else:
            P[i,i//2]     = sym.nsimplify(0.5)
            P[i,(i+1)//2] = sym.nsimplify(0.5)
            
    return P

linear_prolongation_1d(m)

⎡ 1    0    0    0    0    0    0    0    0 ⎤
⎢                                           ⎥
⎢1/2  1/2   0    0    0    0    0    0    0 ⎥
⎢                                           ⎥
⎢ 0    1    0    0    0    0    0    0    0 ⎥
⎢                                           ⎥
⎢ 0   1/2  1/2   0    0    0    0    0    0 ⎥
⎢                                           ⎥
⎢ 0    0    1    0    0    0    0    0    0 ⎥
⎢                                           ⎥
⎢ 0    0   1/2  1/2   0    0    0    0    0 ⎥
⎢                                           ⎥
⎢ 0    0    0    1    0    0    0    0    0 ⎥
⎢                                           ⎥
⎢ 0    0    0   1/2  1/2   0    0    0    0 ⎥
⎢                                           ⎥
⎢ 0    0    0    0    1    0    0    0    0 ⎥
⎢                                           ⎥
⎢ 0    0    0    0   1/2  1/2   0    0    0 ⎥
⎢                                           ⎥
⎢ 0    0    0    0    0    1    0    0    0 ⎥
⎢                                 

In [42]:
injective_restriction_1d(n) @ A @ linear_prolongation_1d(m)

⎡ 1     0     0     0     0     0     0     0     0  ⎤
⎢                                                    ⎥
⎢-1     1    -1                                      ⎥
⎢────   ──   ────   0     0     0     0     0     0  ⎥
⎢   2    2      2                                    ⎥
⎢2⋅h    h    2⋅h                                     ⎥
⎢                                                    ⎥
⎢      -1     1    -1                                ⎥
⎢ 0    ────   ──   ────   0     0     0     0     0  ⎥
⎢         2    2      2                              ⎥
⎢      2⋅h    h    2⋅h                               ⎥
⎢                                                    ⎥
⎢            -1     1    -1                          ⎥
⎢ 0     0    ────   ──   ────   0     0     0     0  ⎥
⎢               2    2      2                        ⎥
⎢            2⋅h    h    2⋅h                         ⎥
⎢                                                    ⎥
⎢                  -1     1    -1                    ⎥
⎢ 0     0 

In [43]:
linear_prolongation_1d(m).T @ A @ linear_prolongation_1d(m)

⎡ 1     0     0     0     0     0     0     0     0  ⎤
⎢                                                    ⎥
⎢-1     1    -1                                      ⎥
⎢────   ──   ────   0     0     0     0     0     0  ⎥
⎢   2    2      2                                    ⎥
⎢2⋅h    h    2⋅h                                     ⎥
⎢                                                    ⎥
⎢      -1     1    -1                                ⎥
⎢ 0    ────   ──   ────   0     0     0     0     0  ⎥
⎢         2    2      2                              ⎥
⎢      2⋅h    h    2⋅h                               ⎥
⎢                                                    ⎥
⎢            -1     1    -1                          ⎥
⎢ 0     0    ────   ──   ────   0     0     0     0  ⎥
⎢               2    2      2                        ⎥
⎢            2⋅h    h    2⋅h                         ⎥
⎢                                                    ⎥
⎢                  -1     1    -1                    ⎥
⎢ 0     0 

## Full weight restriction

In [44]:
def full_weight_restriction_1d(n):
    m = 1 + (n-1)//2
    
    R = sym.zeros(m,n)
    R[0,0]   = 1
    R[-1,-1] = 1
    
    for i in range(1,m-1):
        R[i,2*i-1] = sym.nsimplify(0.25)
        R[i,2*i]   = sym.nsimplify(0.5)
        R[i,2*i+1] = sym.nsimplify(0.25)
    
    return R

full_weight_restriction_1d(n)

⎡1   0    0    0    0    0    0    0    0    0    0    0    0    0    0    0   0⎤
⎢                                                                               ⎥
⎢0  1/4  1/2  1/4   0    0    0    0    0    0    0    0    0    0    0    0   0⎥
⎢                                                                               ⎥
⎢0   0    0   1/4  1/2  1/4   0    0    0    0    0    0    0    0    0    0   0⎥
⎢                                                                               ⎥
⎢0   0    0    0    0   1/4  1/2  1/4   0    0    0    0    0    0    0    0   0⎥
⎢                                                                               ⎥
⎢0   0    0    0    0    0    0   1/4  1/2  1/4   0    0    0    0    0    0   0⎥
⎢                                                                               ⎥
⎢0   0    0    0    0    0    0    0    0   1/4  1/2  1/4   0    0    0    0   0⎥
⎢                                                                               ⎥
⎢0   0    0    0

In [45]:
def full_weight_prolongation_1d(m):
    n = 1 + 2*(m-1)
    
    return full_weight_restriction_1d(n).T

full_weight_prolongation_1d(m)

⎡1   0    0    0    0    0    0    0   0⎤
⎢                                       ⎥
⎢0  1/4   0    0    0    0    0    0   0⎥
⎢                                       ⎥
⎢0  1/2   0    0    0    0    0    0   0⎥
⎢                                       ⎥
⎢0  1/4  1/4   0    0    0    0    0   0⎥
⎢                                       ⎥
⎢0   0   1/2   0    0    0    0    0   0⎥
⎢                                       ⎥
⎢0   0   1/4  1/4   0    0    0    0   0⎥
⎢                                       ⎥
⎢0   0    0   1/2   0    0    0    0   0⎥
⎢                                       ⎥
⎢0   0    0   1/4  1/4   0    0    0   0⎥
⎢                                       ⎥
⎢0   0    0    0   1/2   0    0    0   0⎥
⎢                                       ⎥
⎢0   0    0    0   1/4  1/4   0    0   0⎥
⎢                                       ⎥
⎢0   0    0    0    0   1/2   0    0   0⎥
⎢                                       ⎥
⎢0   0    0    0    0   1/4  1/4   0   0⎥
⎢                                 

In [46]:
injective_restriction_1d(n) @ A @ full_weight_prolongation_1d(m)

⎡1   0     0     0     0     0     0     0    0⎤
⎢                                              ⎥
⎢    1    -1                                   ⎥
⎢0  ────  ────   0     0     0     0     0    0⎥
⎢      2     2                                 ⎥
⎢   2⋅h   4⋅h                                  ⎥
⎢                                              ⎥
⎢   -1     1    -1                             ⎥
⎢0  ────  ────  ────   0     0     0     0    0⎥
⎢      2     2     2                           ⎥
⎢   4⋅h   2⋅h   4⋅h                            ⎥
⎢                                              ⎥
⎢         -1     1    -1                       ⎥
⎢0   0    ────  ────  ────   0     0     0    0⎥
⎢            2     2     2                     ⎥
⎢         4⋅h   2⋅h   4⋅h                      ⎥
⎢                                              ⎥
⎢               -1     1    -1                 ⎥
⎢0   0     0    ────  ────  ────   0     0    0⎥
⎢                  2     2     2               ⎥
⎢               4⋅h 

In [47]:
full_weight_restriction_1d(n) @ A @ linear_prolongation_1d(m)

⎡ 1     0     0     0     0     0     0     0     0  ⎤
⎢                                                    ⎥
⎢-1     1    -1                                      ⎥
⎢────  ────  ────   0     0     0     0     0     0  ⎥
⎢   2     2     2                                    ⎥
⎢4⋅h   2⋅h   4⋅h                                     ⎥
⎢                                                    ⎥
⎢      -1     1    -1                                ⎥
⎢ 0    ────  ────  ────   0     0     0     0     0  ⎥
⎢         2     2     2                              ⎥
⎢      4⋅h   2⋅h   4⋅h                               ⎥
⎢                                                    ⎥
⎢            -1     1    -1                          ⎥
⎢ 0     0    ────  ────  ────   0     0     0     0  ⎥
⎢               2     2     2                        ⎥
⎢            4⋅h   2⋅h   4⋅h                         ⎥
⎢                                                    ⎥
⎢                  -1     1    -1                    ⎥
⎢ 0     0 

That corresponds to the stencil of the discretization with spatial step $2h$, except for the boundary terms. That is still ok because the error and residuals at the boundary are always zero.

In [48]:
full_weight_restriction_1d(n) @ A @ full_weight_prolongation_1d(m)

⎡ 1     0     0     0     0     0     0     0     0  ⎤
⎢                                                    ⎥
⎢-1     1    -1                                      ⎥
⎢────  ────  ────   0     0     0     0     0     0  ⎥
⎢   2     2     2                                    ⎥
⎢4⋅h   4⋅h   8⋅h                                     ⎥
⎢                                                    ⎥
⎢      -1     1    -1                                ⎥
⎢ 0    ────  ────  ────   0     0     0     0     0  ⎥
⎢         2     2     2                              ⎥
⎢      8⋅h   4⋅h   8⋅h                               ⎥
⎢                                                    ⎥
⎢            -1     1    -1                          ⎥
⎢ 0     0    ────  ────  ────   0     0     0     0  ⎥
⎢               2     2     2                        ⎥
⎢            8⋅h   4⋅h   8⋅h                         ⎥
⎢                                                    ⎥
⎢                  -1     1    -1                    ⎥
⎢ 0     0 

## 2D Poisson operator
Same formula on a square grid

In [49]:
def build_poisson_operator_2d(n):
    A = sym.zeros(n*n, n*n)
    
    for i in range(n*n):
        row = i // n
        col = i % n
        
        if row == 0 or row == n-1:
            A[i,i] = 1
        elif col == 0 or col == n-1:
            A[i,i] = 1
        else:
            A[i,i]   =  4 / (h*h)
            A[i,i-1] = -1 / (h*h)
            A[i,i+1] = -1 / (h*h)
            A[i,i-n] = -1 / (h*h)
            A[i,i+n] = -1 / (h*h)
            
    return A


build_poisson_operator_2d(3)

⎡1   0   0   0   0    0   0   0   0⎤
⎢                                  ⎥
⎢0   1   0   0   0    0   0   0   0⎥
⎢                                  ⎥
⎢0   0   1   0   0    0   0   0   0⎥
⎢                                  ⎥
⎢0   0   0   1   0    0   0   0   0⎥
⎢                                  ⎥
⎢   -1      -1   4   -1      -1    ⎥
⎢0  ───  0  ───  ──  ───  0  ───  0⎥
⎢     2       2   2    2       2   ⎥
⎢    h       h   h    h       h    ⎥
⎢                                  ⎥
⎢0   0   0   0   0    1   0   0   0⎥
⎢                                  ⎥
⎢0   0   0   0   0    0   1   0   0⎥
⎢                                  ⎥
⎢0   0   0   0   0    0   0   1   0⎥
⎢                                  ⎥
⎣0   0   0   0   0    0   0   0   1⎦

In [50]:
def injective_restriction_2d(n):
    m = 1 + (n-1)//2
    R = sym.zeros(m*m, n*n)

    for i in range(n):
        for j in range(n):
            if i % 2 == 0 and j % 2 == 0:
                R[(i//2)*m+(j//2),i*n+j] = 1
                
    return R

injective_restriction_2d(5)

⎡1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0⎤
⎢                                                                         ⎥
⎢0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0⎥
⎢                                                                         ⎥
⎢0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0⎥
⎢                                                                         ⎥
⎢0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0⎥
⎢                                                                         ⎥
⎢0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0⎥
⎢                                                                         ⎥
⎢0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0⎥
⎢                                                                         ⎥
⎢0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0⎥
⎢           

In [51]:
def full_weight_restriction_2d(n, diag=True):
    m = 1 + (n-1)//2
    R = sym.zeros(m*m, n*n)

    for i in range(0,n,2):
        for j in range(0,n,2):
            if i == 0 or i == n-1 or j == 0 or j == n-1:
                # boundary point
                R[(i//2)*m+(j//2),i*n+j] = 1
            else:
                # center
                R[(i//2)*m+(j//2),i*n+j] = sym.nsimplify(0.25)
                
                # cardinal neighbours
                R[(i//2)*m+(j//2),i*n+j-1] = sym.nsimplify(0.125)
                R[(i//2)*m+(j//2),i*n+j+1] = sym.nsimplify(0.125)
                R[(i//2)*m+(j//2),(i+1)*n+j] = sym.nsimplify(0.125)
                R[(i//2)*m+(j//2),(i-1)*n+j] = sym.nsimplify(0.125)
                
                if diag:
                    R[(i//2)*m+(j//2),(i+1)*n+j+1] = sym.nsimplify(0.0625)
                    R[(i//2)*m+(j//2),(i+1)*n+j-1] = sym.nsimplify(0.0625)
                    R[(i//2)*m+(j//2),(i-1)*n+j+1] = sym.nsimplify(0.0625)
                    R[(i//2)*m+(j//2),(i-1)*n+j-1] = sym.nsimplify(0.0625)
                
                
    return R

full_weight_restriction_2d(5)

⎡1  0  0  0  0  0   0     0    0    0  0   0    0    0   0  0   0     0    0    0  0
⎢                                                                                   
⎢0  0  1  0  0  0   0     0    0    0  0   0    0    0   0  0   0     0    0    0  0
⎢                                                                                   
⎢0  0  0  0  1  0   0     0    0    0  0   0    0    0   0  0   0     0    0    0  0
⎢                                                                                   
⎢0  0  0  0  0  0   0     0    0    0  1   0    0    0   0  0   0     0    0    0  0
⎢                                                                                   
⎢0  0  0  0  0  0  1/16  1/8  1/16  0  0  1/8  1/4  1/8  0  0  1/16  1/8  1/16  0  0
⎢                                                                                   
⎢0  0  0  0  0  0   0     0    0    0  0   0    0    0   1  0   0     0    0    0  0
⎢                                                                

In [52]:
def linear_prolongation_2d(n):
    m = 1 + (n-1)//2
    R = sym.zeros(n*n, m*m)
    
    for i in range(n):
        for j in range(n):
            linear_index = (i//2)*m+(j//2)
            
            if i % 2 == 1 and j % 2 == 1:
                R[i*n+j,linear_index]     = sym.nsimplify(0.25)
                R[i*n+j,linear_index+1]   = sym.nsimplify(0.25)
                R[i*n+j,linear_index+m]   = sym.nsimplify(0.25)
                R[i*n+j,linear_index+m+1] = sym.nsimplify(0.25)
            elif i % 2 == 1:
                R[i*n+j,linear_index]     = sym.nsimplify(0.5)
                R[i*n+j,linear_index+m]   = sym.nsimplify(0.5)
            elif j % 2 == 1:
                R[i*n+j,linear_index]     = sym.nsimplify(0.5)
                R[i*n+j,linear_index+1]   = sym.nsimplify(0.5)
            else:
                R[i*n+j,linear_index] = 1
            
    return R

linear_prolongation_2d(5)                

⎡ 1    0    0    0    0    0    0    0    0 ⎤
⎢                                           ⎥
⎢1/2  1/2   0    0    0    0    0    0    0 ⎥
⎢                                           ⎥
⎢ 0    1    0    0    0    0    0    0    0 ⎥
⎢                                           ⎥
⎢ 0   1/2  1/2   0    0    0    0    0    0 ⎥
⎢                                           ⎥
⎢ 0    0    1    0    0    0    0    0    0 ⎥
⎢                                           ⎥
⎢1/2   0    0   1/2   0    0    0    0    0 ⎥
⎢                                           ⎥
⎢1/4  1/4   0   1/4  1/4   0    0    0    0 ⎥
⎢                                           ⎥
⎢ 0   1/2   0    0   1/2   0    0    0    0 ⎥
⎢                                           ⎥
⎢ 0   1/4  1/4   0   1/4  1/4   0    0    0 ⎥
⎢                                           ⎥
⎢ 0    0   1/2   0    0   1/2   0    0    0 ⎥
⎢                                           ⎥
⎢ 0    0    0    1    0    0    0    0    0 ⎥
⎢                                 

In [53]:
A = build_poisson_operator_2d(5)

injective_restriction_2d(5) @ A @ full_weight_restriction_2d(5).T

⎡1  0  0  0   0    0  0  0  0⎤
⎢                            ⎥
⎢0  1  0  0   0    0  0  0  0⎥
⎢                            ⎥
⎢0  0  1  0   0    0  0  0  0⎥
⎢                            ⎥
⎢0  0  0  1   0    0  0  0  0⎥
⎢                            ⎥
⎢             1              ⎥
⎢0  0  0  0  ────  0  0  0  0⎥
⎢               2            ⎥
⎢            2⋅h             ⎥
⎢                            ⎥
⎢0  0  0  0   0    1  0  0  0⎥
⎢                            ⎥
⎢0  0  0  0   0    0  1  0  0⎥
⎢                            ⎥
⎢0  0  0  0   0    0  0  1  0⎥
⎢                            ⎥
⎣0  0  0  0   0    0  0  0  1⎦

In [54]:
injective_restriction_2d(5) @ A @ linear_prolongation_2d(5)

⎡1   0    0   0    0    0    0   0    0⎤
⎢                                      ⎥
⎢0   1    0   0    0    0    0   0    0⎥
⎢                                      ⎥
⎢0   0    1   0    0    0    0   0    0⎥
⎢                                      ⎥
⎢0   0    0   1    0    0    0   0    0⎥
⎢                                      ⎥
⎢   -1       -1    2   -1       -1     ⎥
⎢0  ────  0  ────  ──  ────  0  ────  0⎥
⎢      2        2   2     2        2   ⎥
⎢   2⋅h      2⋅h   h   2⋅h      2⋅h    ⎥
⎢                                      ⎥
⎢0   0    0   0    0    1    0   0    0⎥
⎢                                      ⎥
⎢0   0    0   0    0    0    1   0    0⎥
⎢                                      ⎥
⎢0   0    0   0    0    0    0   1    0⎥
⎢                                      ⎥
⎣0   0    0   0    0    0    0   0    1⎦

At most we could use this new stencil, it's definitely worth a try...

In [56]:
full_weight_restriction_2d(5) @ A @ linear_prolongation_2d(5)

⎡  1     0      0     0     0     0      0     0      0  ⎤
⎢                                                        ⎥
⎢  0     1      0     0     0     0      0     0      0  ⎥
⎢                                                        ⎥
⎢  0     0      1     0     0     0      0     0      0  ⎥
⎢                                                        ⎥
⎢  0     0      0     1     0     0      0     0      0  ⎥
⎢                                                        ⎥
⎢ -1    -1     -1    -1     3    -1     -1    -1     -1  ⎥
⎢─────  ────  ─────  ────  ────  ────  ─────  ────  ─────⎥
⎢    2     2      2     2     2     2      2     2      2⎥
⎢16⋅h   8⋅h   16⋅h   8⋅h   4⋅h   8⋅h   16⋅h   8⋅h   16⋅h ⎥
⎢                                                        ⎥
⎢  0     0      0     0     0     1      0     0      0  ⎥
⎢                                                        ⎥
⎢  0     0      0     0     0     0      1     0      0  ⎥
⎢                                                       

In [57]:
full_weight_restriction_2d(5,False) @ A @ linear_prolongation_2d(5)

⎡  1     0      0     0     0     0      0     0      0  ⎤
⎢                                                        ⎥
⎢  0     1      0     0     0     0      0     0      0  ⎥
⎢                                                        ⎥
⎢  0     0      1     0     0     0      0     0      0  ⎥
⎢                                                        ⎥
⎢  0     0      0     1     0     0      0     0      0  ⎥
⎢                                                        ⎥
⎢ -1    -1     -1    -1     3    -1     -1    -1     -1  ⎥
⎢─────  ────  ─────  ────  ────  ────  ─────  ────  ─────⎥
⎢    2     2      2     2     2     2      2     2      2⎥
⎢16⋅h   8⋅h   16⋅h   8⋅h   4⋅h   8⋅h   16⋅h   8⋅h   16⋅h ⎥
⎢                                                        ⎥
⎢  0     0      0     0     0     1      0     0      0  ⎥
⎢                                                        ⎥
⎢  0     0      0     0     0     0      1     0      0  ⎥
⎢                                                       